In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv(r"C:\Users\Shraddha\Downloads\playground-series-s5e6\train.csv")
test = pd.read_csv(r"C:\Users\Shraddha\Downloads\playground-series-s5e6\test.csv")

In [3]:
train.head()

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,0,37,70,36,Clayey,Sugarcane,36,4,5,28-28
1,1,27,69,65,Sandy,Millets,30,6,18,28-28
2,2,29,63,32,Sandy,Millets,24,12,16,17-17-17
3,3,35,62,54,Sandy,Barley,39,12,4,10-26-26
4,4,35,58,43,Red,Paddy,37,2,16,DAP


In [4]:
test.head()

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous
0,750000,31,70,52,Sandy,Wheat,34,11,24
1,750001,27,62,45,Red,Sugarcane,30,14,15
2,750002,28,72,28,Clayey,Ground Nuts,14,15,4
3,750003,37,53,57,Black,Ground Nuts,18,17,36
4,750004,31,55,32,Red,Pulses,13,19,14


In [5]:
print(train.shape)
print(test.shape)

(750000, 10)
(250000, 9)


In [6]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [7]:
y_train = encoder.fit_transform(train['Fertilizer Name'])

In [8]:
train_cat = pd.get_dummies(train[['Soil Type', 'Crop Type']])
test_cat = pd.get_dummies(test[['Soil Type', 'Crop Type']])

In [9]:
test_cat = test_cat.reindex(columns=train_cat.columns,fill_value=0)

In [10]:
train_num = train[['Temparature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous']]
test_num = test[['Temparature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous']]

In [11]:
x = pd.concat([train_num, train_cat], axis=1)
y = train['Fertilizer Name']

In [12]:
from sklearn.model_selection import train_test_split

In [22]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=20)

In [23]:
from sklearn.ensemble import RandomForestClassifier

In [24]:
model = RandomForestClassifier(n_estimators=100,max_depth=12,random_state=42)
model.fit(x_train,y_train)

RandomForestClassifier(max_depth=12, random_state=42)

In [25]:
probs = model.predict_proba(pd.concat([test_num, test_cat], axis=1))

In [26]:
top_3 = np.argsort(probs, axis=1)[:, -3:][:, ::-1]

In [27]:
top_3_labels = encoder.inverse_transform(top_3.ravel()).reshape(top_3.shape)
top_3_strings = [' '.join(row) for row in top_3_labels]

In [28]:
test_ids = test['id']

In [29]:
Predicting_Optimal_Fertilizers = pd.DataFrame({
    'id': test_ids,
    'Fertilizer Name': top_3_strings
})
Predicting_Optimal_Fertilizers.to_csv("Predicting_Optimal_Fertilizers.csv", index=False)
print("Saved")

Saved


In [30]:
from sklearn.metrics import label_ranking_average_precision_score

# Convert y_test labels to numerical values (same encoding used during training)
y_test_encoded = encoder.transform(y_test)

# Predict probabilities on x_test (validation set)
val_probs = model.predict_proba(x_test)

# For label_ranking_average_precision_score, we need a binary matrix for true labels
from sklearn.preprocessing import label_binarize
y_test_bin = label_binarize(y_test_encoded, classes=np.arange(len(encoder.classes_)))

# Compute MAP@3
score = label_ranking_average_precision_score(y_test_bin, val_probs)
print(f"Validation MAP score (approx): {score:.4f}")

Validation MAP score (approx): 0.4034
